In [1]:
from Tonnetz_Select import fromMidiToPCS as fmpc
from Tonnetz_Select import analysisFromCorpus
from TrajectoryCalculationsWithClass import NewTrajectory, TrajectoryLookBefore
from graph_creation import CreateGraph
from FirstNotePosition import PlaceFirstNote
from structural_functions import mergeDicts
import os
import pickle
from itertools import islice, groupby
from operator import itemgetter
import music21 as ms
import pretty_midi as ptm

In [2]:
def pickleSave(dictOfGraphs, Composer_Name) :
    completeName = "Comparison_Results/GraphObjects_new/"+ Composer_Name + ".p"
    with open(completeName, 'wb') as config_dictionary_file:
      # Step 3
      pickle.dump(dictOfGraphs, config_dictionary_file)

In [3]:
def findInstruments(file):
    instruments = []
    time_signatures = []
    s = ms.converter.parse(file)
    for element in s.recurse() :
        if 'TimeSignature' in element.classes :
            time_signatures.append(element.ratioString)
        if 'Instrument' in element.classes :
            x = str(element)
            if x != '' :
                if ': ' in x :
                    newstring = x.split(': ', -1)[1]
                    instruments.append(newstring)
                else :
                    instruments.append(x)
    time_signatures = list(map(itemgetter(0), groupby(time_signatures)))
    return instruments, time_signatures

In [4]:
def TempoAdd(directory):
    s = ptm.PrettyMIDI(directory)
    tempo = s.estimate_tempo()
    return int(tempo)

In [5]:
def GraphOfNewPiece(newPiece, directory):

    # extract the directory and the file extension from the piece
    shortfilename = os.path.splitext(os.path.basename(newPiece))[0]
    # Choose from where to parse (directory, corpus)
    if directory == 'corpus':
        file = ms.corpus.parse(newPiece)
        chordList, Tonnetz = analysisFromCorpus(file)
    # add the instruments
        instruments, time_signatures = findInstruments(newPiece)
        tempo = []
    else:
        if newPiece.endswith(".mid") or newPiece.endswith(".MID"):
            complete_name = directory + '/' + newPiece
            chordList, Tonnetz = fmpc(complete_name)
        # add the instruments
            instruments, time_signatures = findInstruments(complete_name)
            tempo = TempoAdd(complete_name)
    firstPoint = PlaceFirstNote(chordList, Tonnetz)
    # build 2 different trajectories
    trajectoryFut = NewTrajectory(chordList, Tonnetz, firstPoint)
    trajectoryFut.addType('Trajectory With Future')
    trajectoryFut.addInstruments(instruments)
    trajectoryFut.addTime_signature_changes(time_signatures)
    trajectoryFut.addTempo(tempo)
    graphFut = CreateGraph(trajectoryFut)
    graphFut.addName(shortfilename)
    
    trajectoryRec = TrajectoryLookBefore(chordList, Tonnetz, firstPoint)
    trajectoryRec.addType('Recursive Trajectory')
    trajectoryRec.addInstruments(instruments)
    trajectoryRec.addTime_signature_changes(time_signatures)
    trajectoryRec.addTempo(tempo)
    graphRec = CreateGraph(trajectoryRec)
    graphRec.addName(shortfilename)
    
    return graphFut, graphRec

In [6]:
def inputQuestions() : 
    composer = input('Do you know the Composer of this Directory ? ')
    style = input('Do you know the Style of the works in this Directory ? ')
    harmony = input('Do you know the Harmony Style of the works in this Directory ? ')
    return composer, style, harmony

In [7]:
def addingLabels(composer, style, harmony, graph):
    if composer != 'No' :
        graph.addComposer(composer)
    if style != 'No' :
        graph.addStyle(style)
    if harmony != 'No':
        graph.addHarmonyStyle(harmony)

In [8]:
def GetWorksByDirectory(directory, composer, style):
    listOfGraphs = []
#     composer, style, harmony = inputQuestions()
    for file in os.listdir(directory):
        if file.endswith(".mid") or file.endswith(
                ".MID") or file.endswith(".mxl") or file.endswith(".xml"):
            try:
                print("Building Trajectory for ", file)
                graphFut, graphRec = GraphOfNewPiece(file, directory)
#                 addingLabels(composer, style, harmony, graph)
                graphFut.addStyle(style)
                graphFut.addComposer(composer)
                graphRec.addStyle(style)
                graphRec.addComposer(composer)
                listOfGraphs.append([graphFut, graphRec])
            except BaseException:
                print("--> Cannot build Trajectory for ", file)
    return listOfGraphs

In [9]:
def GetWorksByComposer(composerName, composer, style):
    listOfGraphs = []
#     composer, style, harmony = inputQuestions()
    listofWorks = ms.corpus.getComposer(composerName)
    if len(listofWorks) > 60 :
        listofWorks = list(islice(listofWorks, 60))
    if len(listofWorks) > 0:
        for piece in listofWorks:
            try:
                print("Building Trajectory for ", piece)
                graphFut, graphRec = GraphOfNewPiece(piece, 'corpus')
#                 addingLabels(composer, style, harmony, graph)
                graphFut.addStyle(style)
                graphFut.addComposer(composer)
                graphRec.addStyle(style)
                graphRec.addComposer(composer)
                listOfGraphs.append([graphFut, graphRec])
            except BaseException:
                print("--> Cannot build Trajectory for ", piece)
    return listOfGraphs

In [10]:
bach = GetWorksByComposer('bach', 'bach', 'barok')

Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\bach\bwv1.6.mxl
Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\bach\bwv10.7.mxl
Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\bach\bwv101.7.mxl
Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\bach\bwv102.7.mxl
Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\bach\bwv103.6.mxl
Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\bach\bwv104.6.mxl
Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\bach\bwv108.6.mxl
Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Py

In [11]:
beethoven = GetWorksByDirectory('Midi_files/Classical/Beethoven', 'beethoven', 'classical')
beethovenCorpus = GetWorksByComposer('beethoven', 'beethoven', 'classical')

Building Trajectory for  Cnpia523.mid
3793
Building Trajectory for  Concvi1.mid
5604
Building Trajectory for  Concvi23.mid
3140
Building Trajectory for  Coriolan.mid
1890
Building Trajectory for  Leonore1.mid
2331
Building Trajectory for  Leonore2.mid
3357
Building Trajectory for  Leonore3.mid
3696
Building Trajectory for  String Quartet n1 op18 1mov.mid
2612
Building Trajectory for  String Quartet n1 op18 2mov.mid
1238
Building Trajectory for  String Quartet n1 op18 3mov.mid
1518
Building Trajectory for  String Quartet n1 op18 4mov.mid
2640
Building Trajectory for  String Quartet n16 op135 1mov.mid
1499
Building Trajectory for  String Quartet n16 op135 2mov.mid
1521
Building Trajectory for  String Quartet n16 op135 3mov.mid
382
Building Trajectory for  String Quartet n16 op135 4mov.mid
--> Cannot build Trajectory for  String Quartet n16 op135 4mov.mid
Building Trajectory for  String Quartet n2 op18 n2 1mov.mid
1963
Building Trajectory for  String Quartet n2 op18 n2 3mov.mid
1651
Build

In [12]:
chopin = GetWorksByDirectory('Midi_files/Classical/Chopin', 'chopin', 'classical')
chopinCorpus = GetWorksByComposer('chopin', 'chopin', 'classical')

Building Trajectory for  56336a_polonaise_op_53_(nc)smythe.mid
2674
Building Trajectory for  69521a_etude_(aeolian_harp)_op_25_no_1_(nc)smythe.mid
1308
Building Trajectory for  andante_polacca_22_(c)finley.mid
5719
Building Trajectory for  chopin_mazurkas_67_2.mid
591
Building Trajectory for  chopin_polonaise-fantasie_61_(c)lubetsky.mid
3987
Building Trajectory for  chopin_raindrop_prelude.mid
413
Building Trajectory for  chpn-p1.mid
246
Building Trajectory for  chpn-p10.mid
185
Building Trajectory for  chpn-p11.mid
147
Building Trajectory for  chpn-p12.mid
394
Building Trajectory for  chpn-p13.mid
384
Building Trajectory for  chpn-p14.mid
217
--> Cannot build Trajectory for  chpn-p14.mid
Building Trajectory for  chpn-p15.mid
375
Building Trajectory for  chpn-p16.mid
707
Building Trajectory for  chpn-p17.mid
348
Building Trajectory for  chpn-p18.mid
243
Building Trajectory for  chpn-p19.mid
544
Building Trajectory for  chpn-p2.mid
159
Building Trajectory for  chpn-p20.mid
59
Building T

C:\Users\melki\AppData\Local\Programs\Python\Python36-32\lib\site-packages\pretty_midi\pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  RuntimeWarning)


Building Trajectory for  nocturne_15_1_(c)unknown.mid
1008
Building Trajectory for  nocturne_62_2_(c)unknown.mid
984
Building Trajectory for  nocturne_72_1_(c)unknown.mid
748
Building Trajectory for  nocturne_9_1_(c)unknown.mid
1204
Building Trajectory for  nocturne_9_2_(c)unknown.mid
622
Building Trajectory for  nocturne_9_3_(c)unknown.mid
1749
Building Trajectory for  nocturne_c_minor_(c)inoue.mid
538
Building Trajectory for  nocturne_op_9_1_2979_r_(nc)smythe.mid
1615
Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\chopin\mazurka06-2.krn


In [13]:
mozart = GetWorksByDirectory('Midi_files/Classical/Mozart', 'mozart', 'classical')
mozartCorpus = GetWorksByComposer('mozart', 'mozart', 'classical')

Building Trajectory for  Ad_fug.mid
1814
Building Trajectory for  Conc_clarinet-1.mid
4176
Building Trajectory for  Conc_clarinet-2.mid
679
Building Trajectory for  Conc_clarinet-3.mid
2994
Building Trajectory for  mz_333_1.mid
4110
Building Trajectory for  mz_333_2.mid
1434
Building Trajectory for  mz_333_3.mid
2193
Building Trajectory for  mz_570_1.mid
2670
Building Trajectory for  mz_570_2.mid
1020
Building Trajectory for  mz_570_3.mid
1234
Building Trajectory for  piano_sonata_310_1_(c)oguri.mid
2321
Building Trajectory for  piano_sonata_310_2_(c)oguri.mid
2418
Building Trajectory for  piano_sonata_310_3_(c)oguri.mid
1430
Building Trajectory for  piano_sonata_330_1_(c)oguri.mid
2038
Building Trajectory for  piano_sonata_330_2_(c)oguri.mid
790
Building Trajectory for  piano_sonata_330_3_(c)oguri.mid
2006
Building Trajectory for  piano_sonata_331_1_(c)oguri.mid
2601
Building Trajectory for  piano_sonata_331_2_(c)oguri.mid
1428
Building Trajectory for  piano_sonata_331_3_(c)oguri.mid


In [14]:
schumann = GetWorksByDirectory('Midi_files/Classical/Schumann', 'schumann', 'classical')
schumannCorpus = GetWorksByComposer('schumann', 'schumann', 'classical')

Building Trajectory for  By-The-Fireside-Opus-15-(8th-Movement).mid
363
Building Trajectory for  Catch-Me!-Opus-15-(3rd-Movement).mid
297
Building Trajectory for  Child-Falling-Asleep-Opus-15-(12th-Movement).mid
367
Building Trajectory for  Curious-Story-Opus-15-(2nd-Movement).mid
370
Building Trajectory for  Fantasy-Pieces-in-A-Flat-Opus-111-Nr-2.mid
1179
Building Trajectory for  Fantasy-Pieces-in-C-Minor-Opus-111-Nr-1.mid
1445
Building Trajectory for  Fantasy-Pieces-in-C-Minor-Opus-111-Nr-3.mid
918
Building Trajectory for  Fantasy-Pieces-Opus-12-Nr-6.mid
756
Building Trajectory for  Foreign-Land-And-People-Opus-15-(1st-Movement).mid
431
Building Trajectory for  Friendly-Landscape-From-Waldscenen.mid
469
Building Trajectory for  Frightening-Opus-15-(11th-Movement).mid
562
Building Trajectory for  Fruhlingsnacht.mid
1041
Building Trajectory for  Important-Event-Opus-15-(6st-Movement).mid
216
Building Trajectory for  In-The-Evening-Opus-12-Nr-1.mid
1115
Building Trajectory for  In-The-N

In [15]:
palestrina = GetWorksByComposer('palestrina', 'palestrina', 'renaissance')
monteverdi = GetWorksByComposer('monteverdi', 'monteverdi', 'renaissance')

Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\palestrina\Agnus.krn
Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\palestrina\Agnus_00.krn
Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\palestrina\Agnus_01.krn
Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\palestrina\Agnus_02.krn
Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\palestrina\Agnus_03.krn
Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\palestrina\Agnus_04.krn
Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\palestrina\Agnus_05.krn
Building Trajectory for  C:\Us

Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\palestrina\Agnus_I_39.krn
Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\monteverdi\madrigal.3.1.mxl
Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\monteverdi\madrigal.3.1.rntxt
Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\monteverdi\madrigal.3.10.mxl
Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\monteverdi\madrigal.3.10.rntxt
Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\monteverdi\madrigal.3.11.mxl
Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\monteverdi\madrigal.3.

Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\monteverdi\madrigal.4.16.rntxt
Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\monteverdi\madrigal.4.17.mxl
Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\monteverdi\madrigal.4.17.rntxt
Building Trajectory for  C:\Users\melki\AppData\Local\Programs\Python\Python36-32\Lib\site-packages\music21\corpus\monteverdi\madrigal.4.18.mxl


In [16]:
jazz = GetWorksByDirectory('Midi_files/Standard_Jazz/Random', 'Unknown', 'jazz')

Building Trajectory for  A-Getting-Sentimental-Over-You-1.mid
516
Building Trajectory for  A-Taste-Of-Honey.mid
863
Building Trajectory for  After-You-Gone.mid
1547
Building Trajectory for  All-Of-Me-2.mid
2630
Building Trajectory for  All-The-Things-You-Are.mid
2922
[0, 2, 4, 6, 8, 10] dict_items([(0, (-3, 4)), (4, (-2, 4)), (8, (-4, 4))]) (-3, 4) 7 6 3
--> Cannot build Trajectory for  All-The-Things-You-Are.mid
Building Trajectory for  All-The-Way.mid
1383
--> Cannot build Trajectory for  All-The-Way.mid
Building Trajectory for  Alone-Together.mid
874
Building Trajectory for  Antropology.mid
1680
[2, 3, 4, 7, 11] dict_items([(2, (-2, -2)), (4, (-2, -1)), (7, (-1, -1)), (11, (-3, -2))]) (-2, -2) 6 5 4
--> Cannot build Trajectory for  Antropology.mid
Building Trajectory for  Anything-Goes.mid
1172
[6, 2, 4, 0, 8, 10] dict_items([(6, (0, -6)), (2, (-1, -6)), (10, (1, -6))]) (0, -6) 7 6 3
--> Cannot build Trajectory for  Anything-Goes.mid
Building Trajectory for  As-Time-Goes-By.mid
1091

In [17]:
pickleSave(chopin, 'chopin')
pickleSave(chopinCorpus, 'chopinCorpus')

pickleSave(beethoven, 'beethoven')
pickleSave(beethovenCorpus, 'beethovenCorpus')

pickleSave(schumann, 'schumann')
pickleSave(schumannCorpus, 'schumannCorpus')

pickleSave(mozart, 'mozart')
pickleSave(mozartCorpus, 'mozartCorpus')

pickleSave(palestrina, 'palestrina')

pickleSave(monteverdi, 'monteverdi')

pickleSave(bach, 'bach')

pickleSave(jazz, 'jazz')